from https://www.kaggle.com/ruby33421/lgbm-with-new-features

## Intro
Please see Alexey Pronin's kernel (https://www.kaggle.com/graf10a/logistic-regression-with-new-features-feather) to read more on feature engineering and the performance benefit of using feather files.
Alexey Pronin's kernel also references Youri Matiounine's work here: (https://www.kaggle.com/ymatioun/santander-linear-model-with-additional-features) 

The featuring engineering process adds 1000 new features, which means a total of 1200 features for the Santander dataset. The original kernel uses a simple logistic regression for training, which achieves a very good score of 0.896 (AUC). This kernel will use Light GBM model, but instead of using incorporating all the 1K additional features in our model, we will use feature importance to select some of the top engineered features only.

In [1]:
import os
import gc
import matplotlib.pyplot as plt
import time
import shutil
import sklearn
import feather
import numpy as np
import optuna
import pandas as pd
import xgboost as xgb

from functools import partial
from scipy.stats import norm, rankdata
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import roc_curve, auc, roc_auc_score
import lightgbm as lgb
from numpy import sort

Now, let's read the CSV files containing the training and testing data and measure how long it takes.

Train:

In [2]:
path_train = '../input/train.feather'
path_test = '../input/test.feather'

print("Reading train data...")
start = time.time()
train = pd.read_csv('../input/train.csv')
end = time.time()

print("It takes {0:.2f} seconds to read 'train.csv'.".format(end - start))

Reading train data...
It takes 4.77 seconds to read 'train.csv'.


Test:

In [3]:
start = time.time()
print("Reading test data...")
test = pd.read_csv('../input/test.csv')
end = time.time()

print("It takes {0:.2f} seconds to read 'test.csv'.".format(end - start))

Reading test data...
It takes 4.62 seconds to read 'test.csv'.


In [4]:
print("Train: ",train.shape)
print("Test: ", test.shape)

Train:  (200000, 202)
Test:  (200000, 201)


Saving the 'target' and 'ID_code' data.

In [5]:
target = train.pop('target')
train_ids = train.pop('ID_code')
test_ids = test.pop('ID_code')

Saving the number of rows in 'train' for future use.

In [6]:
len_train = len(train)

Merging test and train.

In [7]:
merged = pd.concat([train, test])
merged.shape

(400000, 200)

Removing data we no longer need.

In [8]:
del test, train
gc.collect()

148

Saving the list of original features in a new list `original_features`.

In [9]:
original_features = merged.columns

## Computing new features

In [10]:
for col in merged.columns:
    # Normalize the data, so that it can be used in norm.cdf(), 
    # as though it is a standard normal variable
    merged[col] = ((merged[col] - merged[col].mean()) 
    / merged[col].std()).astype('float32')

    # Square
    merged[col+'^2'] = merged[col] * merged[col]

    # Cube
    merged[col+'^3'] = merged[col] * merged[col] * merged[col]

    # 4th power
    merged[col+'^4'] = merged[col] * merged[col] * merged[col] * merged[col]

    # Cumulative percentile (not normalized)
    merged[col+'_cp'] = rankdata(merged[col]).astype('float32')

    # Cumulative normal percentile
    merged[col+'_cnp'] = norm.cdf(merged[col]).astype('float32')

Getting the list of names of the added features.

In [11]:
new_features = set(merged.columns) - set(original_features)

Normalize the data. Again.

In [12]:
for col in new_features:
    merged[col] = ((merged[col] - merged[col].mean()) 
    / merged[col].std()).astype('float32')

Saving the data to feather files.

In [22]:
path_target = 'target.feather'

path_train_ids = 'train_ids_extra_features.feather'
path_test_ids = 'test_ids_extra_features.feather'

path_train = 'train_extra_features.feather'
path_test = 'test_extra_features.feather'

print("Writing target to a feather files...")
pd.DataFrame({'target' : target.values}).to_feather(path_target)

print("Writing train_ids to a feather files...")
pd.DataFrame({'ID_code' : train_ids.values}).to_feather(path_train_ids)

print("Writing test_ids to a feather files...")
pd.DataFrame({'ID_code' : test_ids.values}).to_feather(path_test_ids)

print("Writing train to a feather files...")
feather.write_dataframe(merged.iloc[:len_train], path_train)

print("Writing test to a feather files...")
feather.write_dataframe(merged.iloc[len_train:], path_test)

Writing target to a feather files...
Writing train_ids to a feather files...
Writing test_ids to a feather files...
Writing train to a feather files...
Writing test to a feather files...


Removing data we no longer need.

In [23]:
del target, train_ids, test_ids, merged
gc.collect()

27

## Loading the data from feather files

Now let's load of these data back into memory. This will help us to illustrate the advantage of using the feather file format.

In [2]:
path_target = 'target.feather'

path_train_ids = 'train_ids_extra_features.feather'
path_test_ids = 'test_ids_extra_features.feather'

path_train = 'train_extra_features.feather'
path_test = 'test_extra_features.feather'

print("Reading target")
start = time.time()
y = feather.read_dataframe(path_target).values.ravel()
end = time.time()

print("{0:5f} sec".format(end - start))

Reading target


/home/sano/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning:

.labels was deprecated in version 0.24.0. Use .codes instead.



0.012558 sec


In [3]:
print("Reading train_ids")
start = time.time()
train_ids = feather.read_dataframe(path_train_ids).values.ravel()
end = time.time()

print("{0:5f} sec".format(end - start))

Reading train_ids
0.010859 sec


In [4]:
print("Reading test_ids")
start = time.time()
test_ids = feather.read_dataframe(path_test_ids).values.ravel()
end = time.time()

print("{0:5f} sec".format(end - start))

Reading test_ids
0.024150 sec


In [5]:
print("Reading training data")

start = time.time()
train = feather.read_dataframe(path_train)
end = time.time()

print("{0:5f} sec".format(end - start))

Reading training data
0.277851 sec


In [6]:
print("Reading testing data")

start = time.time()
test = feather.read_dataframe(path_test)
end = time.time()

print("{0:5f} sec".format(end - start))

Reading testing data
0.259630 sec


In [7]:
print("Total number of features: ",train.shape[1])

Total number of features:  1200


Hopefully now you can see the great advantage of using the feather files: it is blazing fast. Just compare the timings shown above with those measured for the original CSV files: the processed data sets (stored in the feather file format) that we have just loaded are much bigger in size that the original ones (stored in the CSV files) but we can load them in almost no time!

# Logistic regession with the added features.

Now let's finally do some modeling! More specifically, we will build a straighforward logistic regression model to see whether or not we can improve on linear regression result (LB 0.894). 

In [8]:
NFOLDS = 5
RANDOM_STATE = 871972

feature_list = train.columns

test = test[feature_list]

X = train.values.astype(float)
X_test = test.values.astype(float)

folds = StratifiedKFold(n_splits=NFOLDS, shuffle=True, 
                        random_state=RANDOM_STATE)
oof_preds = np.zeros((len(train), 1))
test_preds = np.zeros((len(test), 1))
roc_cv =[]

In [14]:
for fold_, (trn_, val_) in enumerate(folds.split(y, y)):
    print("Current Fold: {}".format(fold_))
    trn_x, trn_y = X[trn_, :], y[trn_]
    val_x, val_y = X[val_, :], y[val_]
    
    clf =  LogisticRegression(solver='lbfgs', max_iter=1500, C=10)

    clf.fit(trn_x, trn_y)

    val_pred = clf.predict_proba(val_x)[:,1]
    test_fold_pred = clf.predict_proba(X_test)[:,1]
    
    roc_cv.append(roc_auc_score(val_y, val_pred))
    
    print("AUC = {}".format(roc_auc_score(val_y, val_pred)))
    oof_preds[val_, :] = val_pred.reshape((-1, 1))
    test_preds += test_fold_pred.reshape((-1, 1))

Current Fold: 0
AUC = 0.8976432005780828
Current Fold: 1
AUC = 0.8969755043464758
Current Fold: 2
AUC = 0.8996586412019945
Current Fold: 3
AUC = 0.8962648030526484
Current Fold: 4
AUC = 0.8950120474162403


In [32]:
submission_logistic = pd.DataFrame({
        "ID_code": test_ids,
        "target": test_preds.reshape(-1)
})
submission_logistic.to_csv('submission_logistic.csv', index=False)

### Feature Importance & Feature Selection

In [ ]:
feature_importance = abs(clf.coef_[0])
sorted_idx = np.argsort(feature_importance)[::-1]

In [ ]:
top_new_features = feature_list[sorted_idx[0:100]]

In [ ]:
train_newf = train[top_new_features]
Orig_feature_list = list(original_features)

In [ ]:
cols = [col for col in train_newf.columns if col not in Orig_feature_list]
len(cols)

In [ ]:
train_newf.columns

In [ ]:
train2 = pd.concat([train[original_features], train[cols]], axis=1)
test2 = test[train2.columns]

## LGBM model with additional features

In [ ]:
fold_n=5
folds = StratifiedKFold(n_splits=fold_n, shuffle=True, random_state=10)

In [ ]:
X = train2.values.astype(float)
X_test = test2.values.astype(float)

In [41]:
params_tuned = {'num_leaves': 9,
         'min_data_in_leaf': 42,
         'objective': 'binary',
         'max_depth': 11,
         'learning_rate': 0.03,
         'boosting': 'gbdt',
         'bagging_freq': 5,
         'bagging_fraction': 0.8,
         'feature_fraction': 0.8201,
         'bagging_seed': 11,
         'reg_alpha': 3,
         'reg_lambda': 5,
         'random_state': 42,
         'metric': 'auc',
         'verbosity': -1,
         'colsample_bytree': 0.7,
#         'subsample': 0.81,
         'min_gain_to_split': 0.02,
#         'min_child_weight': 19.428902804238373,
         'num_threads': 4}

In [42]:
%%time
y_pred_lgb = np.zeros(len(X_test))
for fold_n, (train_index, valid_index) in enumerate(folds.split(X,y)):
    print('Fold', fold_n, 'started at', time.ctime())
    X_train, X_valid = X[train_index, :], X[valid_index, :]
    y_train, y_valid = y[train_index], y[valid_index]
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid)
        
    lgb_model = lgb.train(params_tuned,train_data,num_boost_round=5000,
                    valid_sets = [train_data, valid_data],verbose_eval=300,early_stopping_rounds = 200)
            
    y_pred_lgb += lgb_model.predict(X_test, num_iteration=lgb_model.best_iteration)/5


Fold 0 started at Fri Mar  8 23:12:16 2019
Training until validation scores don't improve for 200 rounds.
[300]	training's auc: 0.876081	valid_1's auc: 0.85122
[600]	training's auc: 0.903321	valid_1's auc: 0.874298
[900]	training's auc: 0.916552	valid_1's auc: 0.884868
[1200]	training's auc: 0.924382	valid_1's auc: 0.890067
[1500]	training's auc: 0.930065	valid_1's auc: 0.893369
[1800]	training's auc: 0.934512	valid_1's auc: 0.895567
[2100]	training's auc: 0.938381	valid_1's auc: 0.896681
[2400]	training's auc: 0.942003	valid_1's auc: 0.897196
[2700]	training's auc: 0.945713	valid_1's auc: 0.897343
[3000]	training's auc: 0.949116	valid_1's auc: 0.897518
Early stopping, best iteration is:
[2850]	training's auc: 0.947458	valid_1's auc: 0.897579
Fold 1 started at Fri Mar  8 23:18:40 2019
Training until validation scores don't improve for 200 rounds.
[300]	training's auc: 0.875468	valid_1's auc: 0.851887


KeyboardInterrupt: 

In [49]:
count_itr=0
def objective_optuna(X, y, trial):
    global count_itr
    print(count_itr, end=' ')
    count_itr += 1
    #最適化するパラメータを指定
    params = {
        'n_estimators': trial.suggest_int('n_estimators',1,1000),
        'gamma': trial.suggest_uniform('gamma',0,1),
        'reg_lambda': trial.suggest_uniform('lambda',0,2),
        'learning_rate': trial.suggest_uniform('learning_rate',0,0.3),
        'max_depth': trial.suggest_int('max_depth', 1, 15),
        'subsample': trial.suggest_uniform('subsample',0,1),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree',0,1),
        'min_child_weight': trial.suggest_uniform('min_child_weight',0,10),
        'tree_method':'gpu_exact'
    }
    
    #モデルを定義
    model = xgb.XGBClassifier(random_state=2,**params)
    
#     (train_X, test_X ,train_y, test_y) = train_test_split(X, y, test_size = 0.3, random_state = 2)
    
#     #3-foldクロスバリデーション
    kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=2)
    scores = cross_validate(model, X=X, y=y, cv=kf, scoring='roc_auc')
    
    return 1.0 - scores['test_score'].mean()

In [ ]:
f = partial(objective_optuna, X, y)
study = optuna.create_study()
study.optimize(f, n_trials=10000)

0 

[I 2019-03-08 23:43:30,762] Finished a trial resulted in value: 0.165576625934182. Current best value is 0.165576625934182 with parameters: {'n_estimators': 511, 'gamma': 0.35291487336113525, 'lambda': 1.3895516004349984, 'learning_rate': 0.17423041322586422, 'max_depth': 9, 'subsample': 0.7319257294591153, 'colsample_bytree': 0.8784437620977481, 'min_child_weight': 8.273633209020229}.


1 

[I 2019-03-09 00:13:00,276] Finished a trial resulted in value: 0.13351448604231597. Current best value is 0.13351448604231597 with parameters: {'n_estimators': 490, 'gamma': 0.9733822084942044, 'lambda': 1.2337653708464957, 'learning_rate': 0.06200839595128288, 'max_depth': 13, 'subsample': 0.39659485727811083, 'colsample_bytree': 0.49382731218251896, 'min_child_weight': 5.543425079888281}.


2 

...

3 

[I 2019-03-09 00:18:27,731] Finished a trial resulted in value: 0.2494588885852348. Current best value is 0.1177019991863214 with parameters: {'n_estimators': 387, 'gamma': 0.9577287900840181, 'lambda': 1.8829687459578166, 'learning_rate': 0.1211109191110216, 'max_depth': 3, 'subsample': 0.04628499940930242, 'colsample_bytree': 0.5230606855073078, 'min_child_weight': 2.1473782298925217}.


4 

[I 2019-03-09 00:53:36,473] Finished a trial resulted in value: 0.16236016581527812. Current best value is 0.1177019991863214 with parameters: {'n_estimators': 387, 'gamma': 0.9577287900840181, 'lambda': 1.8829687459578166, 'learning_rate': 0.1211109191110216, 'max_depth': 3, 'subsample': 0.04628499940930242, 'colsample_bytree': 0.5230606855073078, 'min_child_weight': 2.1473782298925217}.


5 

## Submission File

In [ ]:
submission_lgb = pd.DataFrame({
        "ID_code": test_ids,
        "target": y_pred_lgb
    })
submission_lgb.to_csv('submission_lgb.csv', index=False)